In [45]:
import pandas as pd
from collections import OrderedDict
import warnings

warnings.filterwarnings("ignore")

class gen_reg_code_of_uvm(object):
    def __init__(self):
        self.start_name = "uart"  # the start string of object
        self.reg_block_name = "ral_%s_reg" % self.start_name
        self.table_file = "uart_reg.xlsx"
        self.sv_file_name = "ral_" + self.start_name + "_regmodel_b.sv"
        self.data_df = None
        self.reg_info = OrderedDict()
        self.reg_class_sv_all = []
        
        self.reg_class_info = [
                            "class %s extends uvm_reg;", 
                            "    `uvm_object_utils(%s)",
                            "    virtual function void build();",
                            "        %s = uvm_reg_field::type_id::create(\"%s\");",
                            "        %s.configure(this, %s);",
                            "    function new(string name = \"%s\");", 
                            "    endfunction",
                            "        super.new(name, 32, UVM_NO_COVERAGE);"
                        ]
        
        self.reg_block_info = [
                            "class %s extends uvm_reg_block;",
                            "    rand %s_reg_%s %s;",
                            "    `uvm_object_utils(%s)",
                            "    function new(string name = \"%s\");",
                            "        super.new(name, UVM_NO_COVERAGE);",
                            "    endfunction",
                            "    virtual function void build();",
                            "        default_map = create_map(\"default_map\", 0, 4, UVM_BIG_ENDIAN, 1);",
                            "        %s = %s::type_id::create(\"%s\", null, get_full_name());",
                            "        %s.configure(this, null, \"\");",
                            "        %s.build();",
                            "        default_map.add_reg(%s, %s, %s);"
                        ]
        
        self.run_all()
        
    def run_all(self):
        self.read_data()
        self.get_all_info_to_orderdict()
        self.gen_all_register_code()
        self.gen_all_register_block_code()
        self.write()
        self.print_code()
        
    def read_data(self):
        self.data_df = pd.read_excel(self.table_file, encoding="utf-8")
        self.data_df.fillna("", inplace=True)

    def get_field_info(self, info):
        """
        get filed info to list
        """
        index_e, index_s = info[0][1:-1].split(":")
        index_e, index_s = int(index_e), int(index_s)
        filed_li = [
            info[1],
            str(index_e - index_s + 1),
            str(index_s), "\"" + info[2] + "\"", "0",
            "'h" + format(eval(info[3]), 'X'), "1", "1", "1"
        ]
        return filed_li

    def get_all_info_to_orderdict(self):
        # get all info to orderdict
        for li in self.data_df.values.tolist():
            if li[0]:
                reg_name = li[0]
                self.reg_info[reg_name] = OrderedDict({
                    "name": li[0],
                    "addr": li[1],
                    "field": []
                })
                self.reg_info[reg_name]["field"].append(self.get_field_info(li[2:]))
            else:
                self.reg_info[reg_name]["field"].append(self.get_field_info(li[2:]))
                
    # get one register class code
    def gen_reg_one(self, reg_name):
        reg_name_full = "%s_reg_%s" % (self.start_name, reg_name)
        
        self.reg_class_sv_all.append(self.reg_class_info[0] % reg_name_full)
        self.reg_class_sv_all.append(self.reg_class_info[1] % reg_name_full)
        self.reg_class_sv_all.append("")

        for li in self.reg_info[reg_name]['field']:
            self.reg_class_sv_all.append("    rand uvm_reg_field %s;" % li[0])
        self.reg_class_sv_all.append("")
        
        self.reg_class_sv_all.append(self.reg_class_info[2])
        
        for li in self.reg_info[reg_name]['field']:
            self.reg_class_sv_all.append(self.reg_class_info[3] % (li[0], li[0]))  # create
            self.reg_class_sv_all.append(self.reg_class_info[4] % (li[0], ", ".join(li[1:])))  # configure
            self.reg_class_sv_all.append("")
        
        self.reg_class_sv_all.append(self.reg_class_info[6]) # endfunction
        self.reg_class_sv_all.append("")

        # the new function
        self.reg_class_sv_all.append(self.reg_class_info[5] % reg_name_full)
        self.reg_class_sv_all.append(self.reg_class_info[7])
        self.reg_class_sv_all.append(self.reg_class_info[6])

        self.reg_class_sv_all.append("endclass")
        self.reg_class_sv_all.append("")

    def gen_all_register_code(self):
        for reg_name in self.reg_info.keys():
            #print(reg_name)
            self.gen_reg_one(reg_name)
                
    def gen_all_register_block_code(self):
        self.reg_class_sv_all.append(self.reg_block_info[0] % self.reg_block_name)  # define register block class
        self.reg_class_sv_all.append("")

        # the rand code of all reigister
        for reg_name in self.reg_info.keys():
            self.reg_class_sv_all.append(self.reg_block_info[1] % (self.start_name, reg_name, reg_name))

        self.reg_class_sv_all.append("")
        self.reg_class_sv_all.append(self.reg_block_info[2] % self.reg_block_name)
        self.reg_class_sv_all.append("")

        # the new function
        self.reg_class_sv_all.append(self.reg_block_info[3] % self.reg_block_name)
        self.reg_class_sv_all.append(self.reg_block_info[4])
        self.reg_class_sv_all.append(self.reg_block_info[5])
        self.reg_class_sv_all.append("")

        self.reg_class_sv_all.append(self.reg_block_info[6]) # the build function
        self.reg_class_sv_all.append(self.reg_block_info[7]) # create_map
        self.reg_class_sv_all.append("")

        # all code of add all register
        for reg_name in self.reg_info.keys():
            #print(reg_name)
            self.reg_class_sv_all.append(self.reg_block_info[8] % (reg_name, self.start_name + "_reg_" + reg_name, reg_name))
            self.reg_class_sv_all.append(self.reg_block_info[9] % reg_name)
            self.reg_class_sv_all.append(self.reg_block_info[10] % reg_name)
            self.reg_class_sv_all.append(self.reg_block_info[11] % (reg_name, "'h0" + format(eval(self.reg_info[reg_name]["addr"]), 'X'), "\"RW\""))
            self.reg_class_sv_all.append("")

        self.reg_class_sv_all.append(self.reg_block_info[5])
        self.reg_class_sv_all.append("endclass")

    def write(self):
        with open(self.sv_file_name, "w", encoding="utf-8") as f:
            for line in self.reg_class_sv_all:
                f.write(line + "\n")
                
    def print_code(self):
        for line in self.reg_class_sv_all:
            print(line)

In [46]:
gen_model = gen_reg_code_of_uvm();
gen_model.run_all()

class uart_reg_UARTDR extends uvm_reg;
    `uvm_object_utils(uart_reg_UARTDR)

    rand uvm_reg_field reserved;
    rand uvm_reg_field OE;
    rand uvm_reg_field BE;
    rand uvm_reg_field PE;
    rand uvm_reg_field FE;
    rand uvm_reg_field DATA;

    virtual function void build();
        reserved = uvm_reg_field::type_id::create("reserved");
        reserved.configure(this, 4, 12, "RO", 0, 'h0, 1, 1, 1);

        OE = uvm_reg_field::type_id::create("OE");
        OE.configure(this, 1, 11, "RW", 0, 'h0, 1, 1, 1);

        BE = uvm_reg_field::type_id::create("BE");
        BE.configure(this, 1, 10, "RW", 0, 'h0, 1, 1, 1);

        PE = uvm_reg_field::type_id::create("PE");
        PE.configure(this, 1, 9, "RW", 0, 'h0, 1, 1, 1);

        FE = uvm_reg_field::type_id::create("FE");
        FE.configure(this, 1, 8, "RW", 0, 'h0, 1, 1, 1);

        DATA = uvm_reg_field::type_id::create("DATA");
        DATA.configure(this, 8, 0, "RW", 0, 'h0, 1, 1, 1);

    endfunction

    function new(

        UARTICR = uart_reg_UARTICR::type_id::create("UARTICR", null, get_full_name());
        UARTICR.configure(this, null, "");
        UARTICR.build();
        default_map.add_reg(UARTICR, 'h044, "RW");

    endfunction
endclass
class uart_reg_UARTDR extends uvm_reg;
    `uvm_object_utils(uart_reg_UARTDR)

    rand uvm_reg_field reserved;
    rand uvm_reg_field OE;
    rand uvm_reg_field BE;
    rand uvm_reg_field PE;
    rand uvm_reg_field FE;
    rand uvm_reg_field DATA;

    virtual function void build();
        reserved = uvm_reg_field::type_id::create("reserved");
        reserved.configure(this, 4, 12, "RO", 0, 'h0, 1, 1, 1);

        OE = uvm_reg_field::type_id::create("OE");
        OE.configure(this, 1, 11, "RW", 0, 'h0, 1, 1, 1);

        BE = uvm_reg_field::type_id::create("BE");
        BE.configure(this, 1, 10, "RW", 0, 'h0, 1, 1, 1);

        PE = uvm_reg_field::type_id::create("PE");
        PE.configure(this, 1, 9, "RW", 0, 'h0, 1, 1, 1);

        FE = uvm_reg_field